# Chemistry Drawing Tool

In [3]:
from IPython.display import HTML, display, IFrame
import ipywidgets as widgets
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import QED

In [5]:
# Create Ketcher interface
ketcher_html = """
<div style="display: flex; flex-direction: column; gap: 10px;">
    <iframe id="ketcher_frame" src="https://lifescience.opensource.epam.com/ketcher/demo.html" width="800" height="600" style="border: 1px solid #ccc;"></iframe>
    <div style="display: flex; gap: 10px;">
        <button onclick="getSmiles()" style="padding: 8px 16px; background-color: #4CAF50; color: white; border: none; border-radius: 4px; cursor: pointer;">Get SMILES</button>
        <button onclick="clearKetcher()" style="padding: 8px 16px; background-color: #f44336; color: white; border: none; border-radius: 4px; cursor: pointer;">Clear</button>
    </div>
    <div id="smiles_output" style="padding: 10px; background-color: #f5f5f5; border-radius: 4px;"></div>
</div>

<script>
function getSmiles() {
    const ketcher = document.getElementById('ketcher_frame').contentWindow.ketcher;
    ketcher.getSmiles().then(smiles => {
        document.getElementById('smiles_output').textContent = 'SMILES: ' + smiles;
    });
}

function clearKetcher() {
    const ketcher = document.getElementById('ketcher_frame').contentWindow.ketcher;
    ketcher.clear();
    document.getElementById('smiles_output').textContent = '';
}
</script>
"""

display(HTML(ketcher_html))

In [ ]:
smis = "CC(C)CN(Cc1ccc(cc1)N2CCN(CC2)C(C)=O)[S](=O)(=O)Cc3ccccc3"

mol = Chem.MolFromSmiles(smis)
qed_score = rdkit_qed(mol)
print(f"Original QED: {qed_score}")

# Generate an image from the molecule
img = Draw.MolToImage(mol, size=(300, 300))
display(img)  # this will display the SM image in a Jupyter Notebook

Removing the acyl-piperazine to facilitate exploration of alterantive moeities